<a href="https://colab.research.google.com/github/StephenTGibson/MScDataSci_21-22/blob/main/COMP534-AppliedArtificialIntelligence/NLP_FROM_SCRATCH_CLASSIFYING_NAMES_WITH_A_CHARACTER_LEVEL_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html

In [2]:
from google.colab import files

uploaded = files.upload()

Saving Arabic.txt to Arabic.txt
Saving Chinese.txt to Chinese.txt
Saving Czech.txt to Czech.txt
Saving Dutch.txt to Dutch.txt
Saving English.txt to English.txt
Saving French.txt to French.txt
Saving German.txt to German.txt
Saving Greek.txt to Greek.txt
Saving Irish.txt to Irish.txt
Saving Italian.txt to Italian.txt
Saving Japanese.txt to Japanese.txt
Saving Korean.txt to Korean.txt
Saving Polish.txt to Polish.txt
Saving Portuguese.txt to Portuguese.txt
Saving Russian.txt to Russian.txt
Saving Scottish.txt to Scottish.txt
Saving Spanish.txt to Spanish.txt
Saving Vietnamese.txt to Vietnamese.txt


In [5]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

import unicodedata
import string

In [7]:
def findFiles(path): return glob.glob(path)

print(findFiles('*.txt'))

['Italian.txt', 'English.txt', 'Spanish.txt', 'Irish.txt', 'French.txt', 'Polish.txt', 'Vietnamese.txt', 'Portuguese.txt', 'Scottish.txt', 'Czech.txt', 'Japanese.txt', 'Dutch.txt', 'Greek.txt', 'Chinese.txt', 'German.txt', 'Russian.txt', 'Arabic.txt', 'Korean.txt']


In [9]:


all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

Slusarski


In [11]:
all_categories

['Italian',
 'English',
 'Spanish',
 'Irish',
 'French',
 'Polish',
 'Vietnamese',
 'Portuguese',
 'Scottish',
 'Czech',
 'Japanese',
 'Dutch',
 'Greek',
 'Chinese',
 'German',
 'Russian',
 'Arabic',
 'Korean']

In [12]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [13]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([5, 1, 57])


In [17]:
import torch.nn as nn

In [18]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [19]:
input = letterToTensor('A')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

In [20]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[-2.9636, -2.8803, -2.9505, -3.0174, -2.8848, -2.9028, -3.0106, -2.8795,
         -2.9611, -2.8326, -2.8887, -2.8438, -2.7784, -2.8290, -2.8908, -2.8187,
         -2.8715, -2.8589]], grad_fn=<LogSoftmaxBackward0>)
